# Overview

[Check out the training notebook here](https://www.kaggle.com/ravishah1/cots-faster-rcnn-training-w-tf-2-0-od-api-0-474)

[I also took inspiration from this notebook by Khanh](https://www.kaggle.com/khanhlvg/inference-using-efficientdet-d0-model-tensorflow)

### General Info

This notebook is the training for a faster-rcnn using the TensorFlow Object Detection API. It uses TensorFlow 2.0 API making it eligable for the TensorFlow Performance Prize. It scores a 0.474

The Model I trained was a Faster-RCNN with a Resnet-101 feature extractor. I applied a few augmentations, used a momentum optimizer, and split with a group k-fold on sequence. For more details, see the training notebook linked at the top.

I predict with a dataset with weights generated from a notebook identical to the training notebook above.

In [ ]:
import numpy as np
import os
import sys
import tensorflow as tf
import time

# Import the library that is used to submit the prediction result.
INPUT_DIR = '../input/tensorflow-great-barrier-reef/'
sys.path.insert(0, INPUT_DIR)
import greatbarrierreef

In [ ]:
MODEL_DIR = '../input/cots-fasterrcnn-tfod-api-weights/cots_faster_rcnn_resnet101'
start_time = time.time()
tf.keras.backend.clear_session()
detect_fn_tf_odt = tf.saved_model.load(os.path.join(os.path.join(MODEL_DIR, 'output'), 'saved_model'))
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed time: ' + str(elapsed_time) + 's')

In [ ]:
""" These functions are not currently being used """

def RecoverHE(sceneRadiance):
    for i in range(3):
        sceneRadiance[:, :, i] =  cv2.equalizeHist(sceneRadiance[:, :, i])
    return sceneRadiance

def apply_HE(img_path: str):
    img = cv2.imread(img_path)
    sceneRadiance = RecoverHE(img)
    return sceneRadiance

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path (this can be local or on colossus)

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    #image = Image.fromarray(apply_HE(path))
    
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(io.BytesIO(img_data))
    (im_width, im_height) = image.size
    
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def detect(image_np):
    """Detect COTS from a given numpy image."""

    input_tensor = np.expand_dims(image_np, 0)
    start_time = time.time()
    detections = detect_fn_tf_odt(input_tensor)
    return detections

In [ ]:
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [ ]:
DETECTION_THRESHOLD = 0.50

submission_dict = {
    'id': [],
    'prediction_string': [],
}

for (image_np, sample_prediction_df) in iter_test:
    height, width, _ = image_np.shape
    
    # Run object detection using the TensorFlow model.
    detections = detect(image_np)
    
    # Parse the detection result and generate a prediction string.
    num_detections = detections['num_detections'][0].numpy().astype(np.int32)
    predictions = []
    for index in range(num_detections):
        score = detections['detection_scores'][0][index].numpy()
        if score < DETECTION_THRESHOLD:
            continue

        bbox = detections['detection_boxes'][0][index].numpy()
        y_min = int(bbox[0] * height)
        x_min = int(bbox[1] * width)
        y_max = int(bbox[2] * height)
        x_max = int(bbox[3] * width)
        
        bbox_width = x_max - x_min
        bbox_height = y_max - y_min
        
        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    
    # Generate the submission data.
    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)

    print('Prediction:', prediction_str)